In [1]:
import torch
import torchvision

import numpy as np
import math

In [2]:
mnist_trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ]))
mnist_testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5,), (0.5,))
                             ]))

In [3]:
## Parameters:
n_epochs = 3
batch_size_train = 10000
batch_size_test = 500
log_interval = 500

In [4]:
train_loader = torch.utils.data.DataLoader(mnist_trainset,batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset,batch_size=batch_size_test, shuffle=False)

In [5]:
import papayaclient

In [6]:
class TheModel(torch.nn.Module):

    def __init__(self):
        super(TheModel, self).__init__()

        self.linear1 = torch.nn.Linear(784, 400)
        self.linear2 = torch.nn.Linear(400, 10)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x1 = x.flatten(start_dim = 1)
        return self.linear2(self.relu(self.linear1(x1)))

In [7]:
clients = []
for batchno, (ex_data, ex_labels) in enumerate(train_loader):
    clients.append(papayaclient.PapayaClient(dat = ex_data,
                                            labs = ex_labels,
                                            batch_sz = 500,
                                            num_partners = 5,
                                            model_class = TheModel,
                                            loss_fn = torch.nn.CrossEntropyLoss))

In [8]:
## Train the Nodes
num_epochs_total = 100
num_epochs_per_swap = 5
num_times = (num_epochs_total // num_epochs_per_swap)
for i in range(0, num_times):
    for n in clients:
        for j in range(0, num_epochs_per_swap):
            n.model_train_epoch()
    if i > 1 and i < num_times - 1 :
        for n in clients:
            n.select_partners(3)
        for n in clients:
            for i in range(0, 4) :
                n.update_partner_weights()
        n.average_partners()

In [9]:
for c in clients :
    print(c.logs['stringy'][99])

node2317epoch 99 loss 0.43524304032325745
node4618epoch 99 loss 0.49172699451446533
node1805epoch 99 loss 0.4933609664440155
node2657epoch 99 loss 0.4919908940792084
node4148epoch 99 loss 0.4787170886993408
node3817epoch 99 loss 0.6907057762145996


In [10]:
accuracies = {}
with torch.no_grad():
    for i in clients :
        accuracies_node = []
        for batchno, (ex_data, ex_labels) in enumerate(test_loader) :
            accuracies_node.append(((i.model.forward(ex_data).argmax(dim = 1) == ex_labels).float().mean()).item())
        accuracies[i.node_id] = np.array(accuracies_node).mean()

In [11]:
accuracies

{2317: 0.8178000003099442,
 4618: 0.8174000024795532,
 1805: 0.8187000095844269,
 2657: 0.8220000058412552,
 4148: 0.8220000058412552,
 3817: 0.7893999993801117}